In [1]:
%pip install duckdb
%pip install notebook
%pip install jupysql pandas matplotlib duckdb-engine


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
input_path = "./data/geojson/random.geojson"
input_filename = os.path.basename(input_path)

In [3]:
import duckdb
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 

In [4]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb'

Success


In [5]:
import random
import string

try:
    conn.raw_connection().remove_function('random_string')
except:
    pass

# Define the Python function to generate a random string of length 'n'
def random_string(n: int) -> str:
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Register the function with DuckDB

conn.raw_connection().create_function('random_string', random_string, side_effects=True)

%sql SELECT random_string(10) AS random_string;

Running query in 'duckdb'

random_string
VXNWtdYabd


In [6]:
%sql ROLLBACK

Running query in 'duckdb'

Success


In [7]:
%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH points AS (
    FROM ST_GeneratePoints({min_x: -700, min_y:-700, max_x:700, max_y:700}::BOX_2D, 5_000_000, 42)
    SELECT point
)
FROM points
SELECT
    id: ROW_NUMBER() OVER (ORDER BY point),
    locationX: point.ST_X(),
    locationy: point.ST_Y(),
    description: random_string(250)


Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [8]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb'

count_star()
5000000


In [9]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,5000000,4828504,2500000.5,1443375.817311613,1237261,2503000,3744826,5000000,0.00
locationX,DOUBLE,-699.9998875427991,699.9996812082827,5287392,-0.2554146521200363,404.07585303254444,-353.83172602355194,0.49498490376301524,348.22205053206415,5000000,0.00
locationy,DOUBLE,-699.9998904764652,699.9993891455233,5089354,0.26429169008950704,404.24081035196724,-349.64677076200365,0.9175852397494503,350.3584338837844,5000000,0.00
description,VARCHAR,0005gY288d6oP8JOEsQwJxrXfmCZFi0Zuui0Dy7E2gfLJxYAiBsNsfc1tJ5sZ0WvTzzGnwKdoE6cJB9sATk1jxcdTSvo5I6YwWF4w3zSChJ0KGQQaB8yZv0tvxw8Sg7cu2KNN6JrnwB89kXBWiKoInPcCBVPvtS1KXRgTp5pDH1pL1Wu2HNZpH6byUgdRUeAmSoSK7yhmpHx7bkbQBRdSEeCXl9Jq334rVOjnriX86IVtS10wspKBLZE7D,zzzyPxPom0V8FoQ5EwhLOa28yDWCJpFz4s7yZyU2JT4pwSiz5Vyml7ZkcJ5G4qFIwKthga0bm6x1cmUPemfDRo0dVGhgZlraqJKeo8ciMtPVFHg7NS3zgCCeOI7j1uzO3MnEbSoFeHTAIiJeQKxXxiheFBOSBPFeqK6YhrcnflBqHv2xUUBIbF7uKe5HjBn0NK35XGZZZOO5QhWt6MLNuZowsn83cIPyzez1XgFoZpx1elHuyEfZgbJmVC,4894293,None,None,None,None,None,5000000,0.00


In [10]:
export_filename = input_filename.replace(".geojson", ".csv")
export_path = "./data/csv/" + export_filename

os.makedirs(os.path.dirname(export_path), exist_ok=True)

print(f"Exporting to {export_filename}")
print(f"Exporting to {export_path}")

Exporting to sports.csv
Exporting to ./data/csv/sports.csv


In [11]:
%%sql
COPY (SELECT * FROM point_data_centroid) TO '{{export_path}}' (FORMAT CSV, HEADER);

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count
